In [1]:
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchdiffeq import odeint_adjoint as odeint
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
def split(samp_trajs, train_split=0.6, val_split=0.2):
    
    train_int = np.int64(train_split * samp_trajs.shape[1])  # X% of the data length for training
    val_int = np.int64((train_split + val_split) * samp_trajs.shape[1])  # X% more for validation
    
    samp_trajs_train, samp_trajs_val = (samp_trajs[ :,:train_int, :], samp_trajs[:, train_int:val_int,  :])
    
    return samp_trajs_train, samp_trajs_val

In [3]:
datafilepath = 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/RodentPaws_fps200.npy'
fps200data = np.load(datafilepath)

In [4]:
fps200data.shape

(401, 9900, 8)

In [5]:
def load_data(datafilepath):
    data = np.load(datafilepath)
    traj_tot = data.copy()
    traj_tot = traj_tot[:,1000:9000,:]
    data = data[:, 2000:8000, :]
    
    n = data.shape[1]
    start = 0
    stop = 0.25 + 1

    start_idx = 0
    stop_idx = n
    tot_idx = np.int64(data.shape[1]/120-start)

    orig_ts = np.linspace(start, stop-1, num=tot_idx)
    samp_ts = orig_ts[start_idx:stop_idx]

    orig_trajs = np.zeros((data.shape[0],data.shape[1],data.shape[2]))
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            trajs = data[i,:,j]
            trajs_tot = traj_tot[i,:,j]
            orig_trajs[i,:,j] = (trajs - trajs_tot.mean()) / trajs_tot.std()
            samp_trajs = orig_trajs[:,start_idx:stop_idx,:]
    
    samp_trajs, samp_trajs_val =split(samp_trajs, train_split=0.6, val_split=0.2)
    samp_trajs = samp_trajs.reshape(401*72, 50, 8)
    samp_trajs_val = samp_trajs_val.reshape(401*24, 50, 8)

    return orig_trajs, samp_trajs, orig_ts, samp_ts, samp_trajs_val


In [6]:
orig_trajs, samp_trajs, orig_ts, samp_ts, samp_trajs_val = load_data(datafilepath)

In [7]:
samp_trajs = samp_trajs.reshape(401,3600,8)

In [9]:
if not os.path.exists('Results_pic/Autocorrelation_graph/Tied'):
           os.makedirs('Results_pic/Autocorrelation_graph/Tied')
        
if not os.path.exists('Results_pic/Autocorrelation_graph/Split'):
           os.makedirs('Results_pic/Autocorrelation_graph/Split')

In [8]:
samp_trajs.shape

(401, 3600, 8)

In [11]:
from statsmodels.graphics import tsaplots
import math
import matplotlib.pyplot as plt

        
for i in range(samp_trajs.shape[2]):
    x = samp_trajs[204, :, i]
    fig = tsaplots.plot_acf(x, lags=20, color='r', title='Autocorrelation function')
    plt.axhline(y = 1/math.e , color = 'r', linestyle = '-')
    plt.savefig('./Results_pic/Autocorrelation_graph/Tied/autocorrelation_tied_positionalvalue{}.png'.format(i), dpi=250)
    plt.close()
    
for i in range(samp_trajs.shape[2]):
    x = samp_trajs[215, :, i]
    fig = tsaplots.plot_acf(x, lags=20, color='r', title='Autocorrelation function')
    plt.axhline(y = 1/math.e , color = 'r', linestyle = '-')
    plt.savefig('./Results_pic/Autocorrelation_graph/Split/autocorrelation_split_positionalvalue{}.png'.format(i), dpi=250)
    plt.close()